# IEOR 4703 --  Monte Carlo Simulation for FE

## Pricing deep-out-of-money put via simulaiton w/o and w/ stratified sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from time import time

In [ ]:
# import our BMS price and delta functions
from BMS import BMS_price, BMS_delta
# import ecdf function
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
s0 = 1260
K = 900
sig = 0.45
T = 0.25
r = 0.02
q = 0.05

In [ ]:
# exact solution
call = BMS_price('call', s0, K, r, q, sig, T)
print('Exact solution (call): %3.5f' % call)

put = BMS_price('put', s0, K, r, q, sig, T)
print('Exact solution (put): %3.5f' % put)

Exact solution (call): 356.38714
Exact solution (put): 7.55034


In [ ]:
np.random.seed(915611)
n_sims = 50000

## Naive Monte Carlo

In [ ]:
z = np.random.randn(n_sims)
s1 = s0 * np.exp((r - q - sig*sig/2)*T + sig*np.sqrt(T)*z)
payoff = np.exp(-r*T)*np.maximum(K - s1, 0)
p_hat1 = np.mean(payoff)
var_hat1 = np.var(payoff, ddof=1)
std_err1 = np.sqrt(var_hat1 / n_sims)

In [ ]:
print('Naive MC')
print('Exact solution = %3.5f' % put)
print('p_hat = %3.4f' % p_hat1)
print('std error = %3.4f' % std_err1)

Naive MC
Exact solution = 7.55034
p_hat = 7.4409
std error = 0.1411


## Stratified sampling (sub-optimal)

In [ ]:
# Z will be our stratification variable
#delta = np.array([-np.inf, 0, 1, np.inf])
#delta = np.array([-np.inf, 0, np.inf])
delta = np.array([-np.inf, -3, -1, 0, np.inf])


cdf_delta = sp.stats.norm.cdf(delta)
p = np.diff(cdf_delta)
m = len(p)

# num of samples is proportional to p
ni_sub = np.ceil(p * n_sims).astype(int)

print(ni_sub)

p_hat2 = 0.
var_hat2 = 0.

for i in range(m):
    u = cdf_delta[i] + (cdf_delta[i + 1] - cdf_delta[i]) * np.random.rand(ni_sub[i])
    z = sp.stats.norm.ppf(u)
    s1 = s0 * np.exp((r - q - sig*sig/2)*T + sig*np.sqrt(T)*z)
    payoff = np.exp(-r*T)*np.maximum(K - s1, 0)
    theta_i = np.mean(payoff)
    var_i = np.var(payoff, ddof=1)
    p_hat2 += theta_i * p[i]
    var_hat2 += var_i * p[i] * p[i] / ni_sub[i]
    std_err2 = np.sqrt(var_hat2)

[   68  7866 17068 25000]


In [ ]:
print('Stratified sampling (sub-optimal)')
print('Exact solution = %3.5f' % put)
print('Number of samples per stratum: ' + str(ni_sub))
print('p_hat = %3.5f' % p_hat2)
print('std error = %3.4f' % std_err2)

Stratified sampling (sub-optimal)
Exact solution = 7.55034
Number of samples per stratum: [   68  7866 17068 25000]
p_hat = 7.60816
std error = 0.1103


## Stratified sampling (optimal)

In [ ]:
# pilot program to find ni optimal
sig_i = np.zeros(m)
for i in range(m):
    n_pilot = 1000
    u = cdf_delta[i] + (cdf_delta[i + 1] - cdf_delta[i]) * np.random.rand(n_pilot)
    z = sp.stats.norm.ppf(u)
    s1 = s0 * np.exp((r - q - sig*sig/2)*T + sig*np.sqrt(T)*z)
    payoff = np.exp(-r*T)*np.maximum(K - s1, 0)
    sig_i[i] = np.std(payoff, ddof=1)

# num of samples is proportional to p * sigma
tmp = sig_i * p
ni_opt = np.ceil(n_sims * tmp / np.sum(tmp)).astype(int)
# we simulate at least 10 samples in each stratum
ni_opt = np.maximum(ni_opt, 10)

p_hat3 = 0.
var_hat3 = 0.

for i in range(m):
    u = cdf_delta[i] + (cdf_delta[i + 1] - cdf_delta[i]) * np.random.rand(ni_opt[i])
    print(ni_opt[i])
    z = sp.stats.norm.ppf(u)
    s1 = s0 * np.exp((r - q - sig*sig/2)*T + sig*np.sqrt(T)*z)
    payoff = np.exp(-r*T)*np.maximum(K- s1, 0)
    theta_i = np.mean(payoff)
    var_i = np.var(payoff, ddof=1)
    p_hat3 += theta_i * p[i]
    var_hat3 += var_i * p[i] * p[i] / ni_opt[i]
    std_err3 = np.sqrt(var_hat3)

229
49772
10
10


In [ ]:
print('Stratified sampling (optimal)')
print('Exact solution = %3.4f' % put)
print('Number of samples per stratum: ' + str(ni_opt))
print('p_hat = %3.5f' % p_hat3)
print('std error = %3.5f' % std_err3)

Stratified sampling (optimal)
Exact solution = 7.5503
Number of samples per stratum: [  229 49772    10    10]
p_hat = 7.53866
std error = 0.04397


### Comparison of three (3) methods

In [ ]:
print('Exact solution = %3.4f' % put)
print('Number of samples = %i' % n_sims)
print('Naive MC: %3.4f (%3.4f)' % (p_hat1, std_err1))
print('Stratified sampling (sub-optimal): %3.4f (%3.4f)' % (p_hat2, std_err2))
print('Allocation of samples (sub-optimal): ' + str(ni_sub))
print('Stratified sampling (optimal): %3.4f (%3.4f)' % (p_hat3, std_err3))
print('Allocation of samples (optimal): ' + str(ni_opt))

Exact solution = 7.5503
Number of samples = 50000
Naive MC: 7.4409 (0.1411)
Stratified sampling (sub-optimal): 7.6082 (0.1103)
Allocation of samples (sub-optimal): [   68  7866 17068 25000]
Stratified sampling (optimal): 7.5387 (0.0440)
Allocation of samples (optimal): [  229 49772    10    10]
